In [221]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# import plotly.express as px 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (accuracy_score, 
                            confusion_matrix, 
                            classification_report,
                            f1_score,
                            plot_confusion_matrix,
                            precision_recall_curve,
                            precision_score,
                            recall_score)

In [173]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [174]:
mass = pd.read_csv('../data/mass_mobile.csv')

In [175]:
mass.drop(columns=['Unnamed: 0'], inplace=True)

In [176]:
mass.head(2)

,id,country,ccode,region,protestnumber,protesterviolence,location,protesteridentity,sources,notes,...,social_restrictions,start_date,end_date,target,notes_clean,neg,neu,pos,compound,protest_duration
0,201990001,Canada,20,North America,1,0.0,national,unspecified,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,...,0,1990-01-15,1990-01-15,"[0, 0, 0, 0, 1, 0, 0]",canada s railway passenger system was finally ...,0.087,0.913,0.0,-0.8176,1
1,201990002,Canada,20,North America,2,0.0,"Montreal, Quebec",unspecified,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,...,0,1990-06-25,1990-06-25,"[0, 0, 0, 0, 1, 0, 0]",protestors were only identified as young peopl...,0.000,1.000,0.0,0.0000,1


In [178]:
# sorted(list(mass.columns))

In [179]:
# mass.info(verbose=True)

In [180]:
def evaluate_model(y_true, y_preds):
    
    '''
    DOCSTRING HERE
    '''
    
    f1 = f1_score(y_true, y_preds)
    
    precision = precision_score(y_true, y_preds)
    
    recall = recall_score(y_true, y_preds)

    # add accuracy

    return f1, precision, recall

In [181]:
# def build_confusion_matrix(y_true, y_preds, title):
    
#     tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
#     disp = plot_confusion_matrix(model, 
#                                  X_test_sc, 
#                                  y_test, 
# #                                  display_labels=
#                                  cmap="Blues")
    
#     disp.ax_.set_title(title)
#     print(title)
#     print(disp.confusion_matrix)
    
#     plt.show
    
#     return

In [182]:
def response_prediction_columns(model, df, features):
    
    ss = StandardScaler()
    
    X = features
    X_sc = ss.fit_transform(X)
    
    predicted_column = model.predict(X_sc)
    probability_column = model.predict_proba(X_sc)[:,1]
    
    return predicted_column, probability_column

In [183]:
def state_response_predictor(df, features, target, model):
    
    X = features 
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

    ss = StandardScaler()

    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    model.fit(X_train_sc, y_train)
    
    y_preds_train = model.predict(X_train_sc)
    y_preds_test = model.predict(X_test_sc)
    
    f1_score_train, precision_train, recall_train = evaluate_model(y_train, y_preds_train)
    f1_score_test, precision_test, recall_test = evaluate_model(y_test, y_preds_test)
    
#     print(f'{target}- F1_score for {model} model, train set: {f1_score_train}')
#     print(f'{target}- F1_score for {model} model, test set: {f1_score_test}')
#     print(f'{target}- Precision for {model} model, train set: {precision_train}')
#     print(f'{target}- Precision for {model} model, test set: {precision_test}')
#     print(f'{target}- Recall for {model} model, train set: {recall_train}')
#     print(f'{target}- Recall for {model} model, teset set: {recall_test}')
    
    predicted_column, probability_column = response_prediction_columns(model, df, features)
    
#     print(model)

#     build_confusion_matrix(y_test, y_preds_test)

    return predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test  

Creating two new columns that consolidate the three violent state responses -- `beatings, shootings, killings`.
+ `violent_response`: indicates whether or not a violdnt state response was used. (0 for no, 1 for yes)
+ `violent_count`: indicates number (0,1,2,3) of violent ressponses that were recorded. (I anticipate this will be used just to judge collinearity/dependence.)

_NB: Not yet dropping the original three columns._

In [184]:
mass['violent_response'] = 0 
# mass.loc[ (mass['beatings'] == 1 | mass['shootings'] == 1 | mass['killings'] == 1), 'violent_response' ] = 1
mass.loc[mass['beatings'] == 1, 'violent_response'] = 1
mass.loc[mass['shootings'] == 1, 'violent_response'] = 1
mass.loc[mass['killings'] == 1, 'violent_response'] = 1

In [185]:
mass['violent_count'] = mass['beatings'] + mass['shootings'] + mass['killings']

In [186]:
# checking whether the new column looks good 
mass[['beatings', 'shootings','killings', 'violent_response', 'violent_count']]

,beatings,shootings,killings,violent_response,violent_count
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
15174,0,1,0,1,1
15175,0,1,1,1,2
15176,0,0,0,0,0
15177,0,0,0,0,0


In [194]:
mass[ mass['violent_count'] > 1][['beatings', 'shootings','killings', 'violent_response']].count()

beatings            604
shootings           604
killings            604
violent_response    604
dtype: int64

_Saving and pickling `mass` DataFrame that now has `violent_response` and `violent_count` columns. 

In [192]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [191]:
mass.to_csv('../data/03_mass_violent_response.csv', index=False)

In [193]:
mass.to_pickle('../data/03_mass_violent_response.pickle')

In [195]:
features = mass.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
           'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
           'end_date', 'target', 'arrests', 'accomodation', 'beatings', 'crowddispersal', 'ignore', 'killings', 'shootings', 
            'violent_response', 'violent_count'])

LogisticRegression_1 = LogisticRegression(solver='lbfgs', max_iter=1000)

In [196]:
# (mass.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
#            'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
#            'end_date', 'target', 'arrests', 'accomodation', 'beatings',
#            'crowddispersal', 'ignore', 'killings', 'shootings'])).columns

**Adding `predicted` and `probability` columns for each each response to the `mass` DF using `state_response_predictor` function.** 

Responses: `'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'`

In [197]:
mass['arrests_predicted'], mass['arrests_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'arrests', LogisticRegression_1)

In [198]:
mass['accomodation_predicted'], mass['accomodation_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'accomodation', LogisticRegression_1)

In [200]:
mass['beatings_predicted'], mass['beatings_probability'], f1_score_train, \
f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'beatings', LogisticRegression_1)

In [201]:
mass['crowddispersal_predicted'], mass['crowddispersal_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'crowddispersal', LogisticRegression_1)

In [202]:
mass['ignore_predicted'], mass['ignore_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'ignore', LogisticRegression_1)

In [203]:
mass['killings_predicted'], mass['killings_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'killings', LogisticRegression_1)

In [204]:
mass['shootings_predicted'], mass['shootings_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'shootings', LogisticRegression_1)

In [199]:
# looking at where accomodation is '1' and it was also correctly predicted 

mass[ (mass['accomodation'] == mass['accomodation_predicted']) & mass['accomodation']==1 ][['accomodation', 'accomodation_predicted', 'accomodation_probability']]

,accomodation,accomodation_predicted,accomodation_probability
451,1,1,0.921425
1438,1,1,0.609061
1469,1,1,0.550764
1679,1,1,0.596815
2061,1,1,0.540708
...,...,...,...
14736,1,1,0.609921
14737,1,1,0.840545
15133,1,1,0.524657
15149,1,1,0.531713


In [246]:
# predictions_df = mass[['id','arrests', 'arrests_predicted', 'arrests_probability', 'accomodation', 'accomodation_predicted', 'accomodation_probability', 
#       'beatings', 'beatings_predicted', 'beatings_probability', 'crowddispersal', 'crowddispersal_predicted', 'crowddispersal_probability', 
#       'ignore', 'ignore_predicted', 'ignore_probability', 'killings', 'killings_predicted', 'killings_probability',
#       'killings', 'killings_predicted', 'killings_probability']].copy()

In [247]:
# predictions_df.head()

In [207]:
# predictions_df.to_csv('../data/04_predictions_only.csv', index=False)

Adding `predicted` and `probability` columns for a violent response of any type (beatings/shootings/killigs). 

In [208]:
mass['violence_predicted'], mass['violence_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'violent_response', LogisticRegression_1)

In [215]:
model_performance_dict = {
    'target':[],
    'majority_class':[],
    'majority_percent':[],
#     'model':[],
    'f1_score_train':[],
    'f1_score_test':[],
    'precision_train':[],
    'precision_test':[],
    'recall_train':[],
    'recall_test':[] 
}

 
targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings', 'violent_response']    
    
for target in targets:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features, 
                                                                                                                                target, 
                                                                                                                                LogisticRegression_1)
    
    model_performance_dict['target'].append(target)
    model_performance_dict['majority_class'].append(mass[target].value_counts(normalize=True).idxmax())
    model_performance_dict['majority_percent'].append(max(mass[target].value_counts(normalize=True)))
#     model_performance_dict['model'].append(model)
    model_performance_dict['f1_score_train'].append(f1_score_train)
    model_performance_dict['f1_score_test'].append(f1_score_test)
    model_performance_dict['precision_train'].append(precision_train)
    model_performance_dict['precision_test'].append(precision_test)
    model_performance_dict['recall_train'].append(recall_train)
    model_performance_dict['recall_test'].append(recall_test)
    
print(model_performance_dict)
    
model_performance_df = pd.DataFrame(model_performance_dict)
model_performance_df.round(4)

{'target': ['arrests', 'accomodation', 'beatings', 'crowddispersal', 'ignore', 'killings', 'shootings', 'violent_response'], 'majority_class': [0, 0, 0, 0, 1, 0, 0, 0], 'majority_percent': [0.8588181039594176, 0.8997957704723631, 0.9472956057711311, 0.6865406153238026, 0.5434481849924238, 0.9457144739442651, 0.9386652612161539, 0.8753541076487252], 'f1_score_train': [0.24430264357338197, 0.09939759036144577, 0.17173051519154559, 0.6446232626188734, 0.7954094644375178, 0.24191616766467064, 0.24093816631130063, 0.39802336028751123], 'f1_score_test': [0.2722117202268431, 0.06984126984126984, 0.15568862275449102, 0.624173180998196, 0.7837535014005603, 0.1958762886597938, 0.2, 0.35976789168278533], 'precision_train': [0.5690021231422505, 0.6470588235294118, 0.65, 0.7299536116633533, 0.745848279527036, 0.6121212121212121, 0.5947368421052631, 0.6429608127721336], 'precision_test': [0.6605504587155964, 0.55, 0.5416666666666666, 0.7188365650969529, 0.7324607329842932, 0.475, 0.48936170212765956

,target,majority_class,majority_percent,f1_score_train,f1_score_test,precision_train,precision_test,recall_train,recall_test
0,arrests,0,0.8588,0.2443,0.2722,0.5690,0.6606,0.1555,0.1714
1,accomodation,0,0.8998,0.0994,0.0698,0.6471,0.5500,0.0538,0.0373
2,beatings,0,0.9473,0.1717,0.1557,0.6500,0.5417,0.0989,0.0909
3,crowddispersal,0,0.6865,0.6446,0.6242,0.7300,0.7188,0.5772,0.5515
4,ignore,1,0.5434,0.7954,0.7838,0.7458,0.7325,0.8520,0.8428
5,killings,0,0.9457,0.2419,0.1959,0.6121,0.4750,0.1507,0.1234
6,shootings,0,0.9387,0.2409,0.2000,0.5947,0.4894,0.1511,0.1257
7,violent_response,0,0.8754,0.3980,0.3598,0.6430,0.5741,0.2882,0.2620


In [216]:
mass['arrests'].value_counts(normalize=True)

0    0.858818
1    0.141182
Name: arrests, dtype: float64

In [217]:
mass['arrests'].value_counts(normalize=True).idxmax()

0

In [218]:
max(mass['arrests'].value_counts(normalize=True))

0.8588181039594176

In [219]:
# mass.info(verbose=True)

_Saving `mass` DataFrame again now that it has the viole

_Re-running `state_response_predictor` function on dataframe without dummified country columns._ 

In [224]:
mass_no_country = mass.copy()

In [226]:
# mass_no_country.info(verbose=1)

Cell below drops the country columns from `mass_no_country` (created in the above cell) but I minimized it for length/scrolling reasons. 

In [227]:
mass_no_country = mass_no_country.drop(columns=[ 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'country_Croatia',
 'country_Cuba',
 'country_Cyprus',
 'country_Czech Republic',
 'country_Czechoslovakia',
 'country_Denmark',
 'country_Djibouti',
 'country_Dominican Republic',
 'country_Ecuador',
 'country_Egypt',
 'country_El Salvador',
 'country_Equatorial Guinea',
 'country_Eritrea',
 'country_Estonia',
 'country_Ethiopia',
 'country_Finland',
 'country_France',
 'country_Gabon',
 'country_Gambia',
 'country_Georgia',
 'country_Germany',
 'country_Germany East',
 'country_Germany West',
 'country_Ghana',
 'country_Greece',
 'country_Guatemala',
 'country_Guinea',
 'country_Guinea-Bissau',
 'country_Guyana',
 'country_Haiti',
 'country_Honduras',
 'country_Hungary',
 'country_India',
 'country_Indonesia',
 'country_Iran',
 'country_Iraq',
 'country_Ireland',
 'country_Italy',
 'country_Ivory Coast',
 'country_Jamaica',
 'country_Japan',
 'country_Jordan',
 'country_Kazakhstan',
 'country_Kenya',
 'country_Kosovo',
 'country_Kuwait',
 'country_Kyrgyzstan',
 'country_Laos',
 'country_Latvia',
 'country_Lebanon',
 'country_Lesotho',
 'country_Liberia',
 'country_Libya',
 'country_Lithuania',
 'country_Luxembourg',
 'country_Macedonia',
 'country_Madagascar',
 'country_Malawi',
 'country_Malaysia',
 'country_Mali',
 'country_Mauritania',
 'country_Mauritius',
 'country_Mexico',
 'country_Moldova',
 'country_Mongolia',
 'country_Montenegro',
 'country_Morocco',
 'country_Mozambique',
 'country_Myanmar',
 'country_Namibia',
 'country_Nepal',
 'country_Netherlands',
 'country_Nicaragua',
 'country_Niger',
 'country_Nigeria',
 'country_North Korea',
 'country_Norway',
 'country_Oman',
 'country_Pakistan',
 'country_Panama',
 'country_Papua New Guinea',
 'country_Paraguay',
 'country_Peru',
 'country_Philippines',
 'country_Poland',
 'country_Portugal',
 'country_Qatar',
 'country_Romania',
 'country_Russia',
 'country_Rwanda',
 'country_Saudi Arabia',
 'country_Senegal',
 'country_Serbia',
 'country_Serbia and Montenegro',
 'country_Sierra Leone',
 'country_Singapore',
 'country_Slovak Republic',
 'country_Slovenia',
 'country_Somalia',
 'country_South Africa',
 'country_South Korea',
 'country_South Sudan',
 'country_Spain',
 'country_Sri Lanka',
 'country_Sudan',
 'country_Suriname',
 'country_Swaziland',
 'country_Sweden',
 'country_Switzerland',
 'country_Syria',
 'country_Taiwan',
 'country_Tajikistan',
 'country_Tanzania',
 'country_Thailand',
 'country_Timor Leste',
 'country_Togo',
 'country_Tunisia',
 'country_Turkey',
 'country_Turkmenistan',
 'country_USSR',
 'country_Uganda',
 'country_Ukraine',
 'country_United Arab Emirate',
 'country_United Kingdom',
 'country_Uruguay',
 'country_Uzbekistan',
 'country_Venezuela',
 'country_Vietnam',
 'country_Yemen',
 'country_Yugoslavia',
 'country_Zambia',
 'country_Zimbabwe' ])

In [234]:
# mass_no_country.info()

In [235]:
features = mass_no_country.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
           'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
           'end_date', 'target', 'arrests', 'accomodation', 'beatings', 'crowddispersal', 'ignore', 'killings', 'shootings', 
            'violent_response', 'violent_count'])

LogisticRegression_1 = LogisticRegression(solver='lbfgs', max_iter=1000)

In [236]:
# (mass.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
#            'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
#            'end_date', 'target', 'arrests', 'accomodation', 'beatings',
#            'crowddispersal', 'ignore', 'killings', 'shootings'])).columns

_Adding `predicted` and `probability` columns to the `mass_no_country` DataFrame. 

In [237]:
mass_no_country['arrests_predicted'], mass_no_country['arrests_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'arrests', LogisticRegression_1)

In [238]:
mass_no_country['accomodation_predicted'], mass_no_country['accomodation_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'accomodation', LogisticRegression_1)

In [239]:
mass_no_country['beatings_predicted'], mass_no_country['beatings_probability'], f1_score_train, \
f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'beatings', LogisticRegression_1)

In [240]:
mass_no_country['crowddispersal_predicted'], mass_no_country['crowddispersal_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'crowddispersal', LogisticRegression_1)

In [241]:
mass_no_country['ignore_predicted'], mass_no_country['ignore_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'ignore', LogisticRegression_1)

In [242]:
mass_no_country['killings_predicted'], mass_no_country['killings_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'killings', LogisticRegression_1)

In [243]:
mass_no_country['shootings_predicted'], mass_no_country['shootings_probability'], \
f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass_no_country, features, 'shootings', LogisticRegression_1)

In [244]:
# predictions_df = mass[['id','arrests', 'arrests_predicted', 'arrests_probability', 'accomodation', 'accomodation_predicted', 'accomodation_probability', 
#       'beatings', 'beatings_predicted', 'beatings_probability', 'crowddispersal', 'crowddispersal_predicted', 'crowddispersal_probability', 
#       'ignore', 'ignore_predicted', 'ignore_probability', 'killings', 'killings_predicted', 'killings_probability',
#       'killings', 'killings_predicted', 'killings_probability']].copy()

In [245]:
# looking at where accomodation is '1' and it was also correctly predicted 
# in mass_no_country DF to see whether it helped 

mass_no_country[ (mass_no_country['accomodation'] == mass_no_country['accomodation_predicted']) & mass_no_country['accomodation']==1 ][['accomodation', 'accomodation_predicted', 'accomodation_probability']]

,accomodation,accomodation_predicted,accomodation_probability
451,1,1,0.947864
1191,1,1,0.523000
1438,1,1,0.649603
1447,1,1,0.535435
1450,1,1,0.611238
...,...,...,...
15144,1,1,0.573000
15149,1,1,0.516305
15152,1,1,0.601844
15156,1,1,0.568081


Creating a new `model_performance` DF for the model run on `mass_no_country`.

In [254]:
model_performance_dict_nc = {
    'target':[],
    'majority_class':[],
    'majority_percent':[],
    'f1_score_train':[],
    'f1_score_test':[],
    'precision_train':[],
    'precision_test':[],
    'recall_train':[],
    'recall_test':[] 
}

 
targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings', 'violent_response']    
    
for target in targets:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features, 
                                                                                                                                target, 
                                                                                                                                LogisticRegression_1)
    
    model_performance_dict_nc['target'].append(target)
    model_performance_dict_nc['majority_class'].append(mass[target].value_counts(normalize=True).idxmax())
    model_performance_dict_nc['majority_percent'].append(max(mass[target].value_counts(normalize=True)))
    model_performance_dict_nc['f1_score_train'].append(f1_score_train)
    model_performance_dict_nc['f1_score_test'].append(f1_score_test)
    model_performance_dict_nc['precision_train'].append(precision_train)
    model_performance_dict_nc['precision_test'].append(precision_test)
    model_performance_dict_nc['recall_train'].append(recall_train)
    model_performance_dict_nc['recall_test'].append(recall_test)
    
print(model_performance_dict)
    
model_performance_df_nc = pd.DataFrame(model_performance_dict_nc)
# model_performance_df_nc.round(4)

{'target': ['arrests', 'accomodation', 'beatings', 'crowddispersal', 'ignore', 'killings', 'shootings', 'violent_response'], 'majority_class': [0, 0, 0, 0, 1, 0, 0, 0], 'majority_percent': [0.8588181039594176, 0.8997957704723631, 0.9472956057711311, 0.6865406153238026, 0.5434481849924238, 0.9457144739442651, 0.9386652612161539, 0.8753541076487252], 'f1_score_train': [0.24430264357338197, 0.09939759036144577, 0.17173051519154559, 0.6446232626188734, 0.7954094644375178, 0.24191616766467064, 0.24093816631130063, 0.39802336028751123], 'f1_score_test': [0.2722117202268431, 0.06984126984126984, 0.15568862275449102, 0.624173180998196, 0.7837535014005603, 0.1958762886597938, 0.2, 0.35976789168278533], 'precision_train': [0.5690021231422505, 0.6470588235294118, 0.65, 0.7299536116633533, 0.745848279527036, 0.6121212121212121, 0.5947368421052631, 0.6429608127721336], 'precision_test': [0.6605504587155964, 0.55, 0.5416666666666666, 0.7188365650969529, 0.7324607329842932, 0.475, 0.48936170212765956

### Includes country rows 

In [257]:
model_performance_df.round(4)

,target,majority_class,majority_percent,f1_score_train,f1_score_test,precision_train,precision_test,recall_train,recall_test
0,arrests,0,0.8588,0.2443,0.2722,0.5690,0.6606,0.1555,0.1714
1,accomodation,0,0.8998,0.0994,0.0698,0.6471,0.5500,0.0538,0.0373
2,beatings,0,0.9473,0.1717,0.1557,0.6500,0.5417,0.0989,0.0909
3,crowddispersal,0,0.6865,0.6446,0.6242,0.7300,0.7188,0.5772,0.5515
4,ignore,1,0.5434,0.7954,0.7838,0.7458,0.7325,0.8520,0.8428
5,killings,0,0.9457,0.2419,0.1959,0.6121,0.4750,0.1507,0.1234
6,shootings,0,0.9387,0.2409,0.2000,0.5947,0.4894,0.1511,0.1257
7,violent_response,0,0.8754,0.3980,0.3598,0.6430,0.5741,0.2882,0.2620


### Does not include country rows (all countries dropped) 

In [258]:
model_performance_df_nc.round(4)

,target,majority_class,majority_percent,f1_score_train,f1_score_test,precision_train,precision_test,recall_train,recall_test
0,arrests,0,0.8588,0.3124,0.3134,0.5814,0.6014,0.2136,0.2119
1,accomodation,0,0.8998,0.1514,0.1062,0.5373,0.4091,0.0881,0.0610
2,beatings,0,0.9473,0.2638,0.1720,0.5833,0.3721,0.1705,0.1119
3,crowddispersal,0,0.6865,0.6515,0.6318,0.7124,0.7075,0.6002,0.5707
4,ignore,1,0.5434,0.7951,0.7832,0.7510,0.7370,0.8447,0.8355
5,killings,0,0.9457,0.3021,0.2170,0.5781,0.3966,0.2045,0.1494
6,shootings,0,0.9387,0.3112,0.2375,0.5529,0.3974,0.2166,0.1694
7,violent_response,0,0.8754,0.4028,0.3925,0.6214,0.5833,0.2980,0.2958


In [261]:
mass_no_country.columns

Index(['id', 'country', 'ccode', 'region', 'protestnumber',
       'protesterviolence', 'location', 'protesteridentity', 'sources',
       'notes', 'protester_id_type', 'partipants_number',
       'protest_size_category', 'pop_male', 'pop_female', 'pop_total',
       'pop_density', 'prosperity_2020', 'region_Africa', 'region_Asia',
       'region_Central America', 'region_Europe', 'region_MENA',
       'region_North America', 'region_Oceania', 'region_South America',
       'protest_size_category_1,000-4,999',
       'protest_size_category_10,000-100,000', 'protest_size_category_100-999',
       'protest_size_category_5,000-9,999', 'protest_size_category_50-99',
       'protest_size_category_Less than 50',
       'protest_size_category_Over 100,000',
       'protester_id_type_civil_human_rights',
       'protester_id_type_ethnic_group', 'protester_id_type_locals_residents',
       'protester_id_type_pensioners_retirees',
       'protester_id_type_political_group', 'protester_id_type_pr

_Dropping all prediction columns from `mass_no_country` so that I can pickle it and use it in another notebook._

In [263]:
mass_no_country.drop(columns=['arrests_predicted', 'arrests_probability', 'accomodation_predicted',
       'accomodation_probability', 'beatings_predicted',
       'beatings_probability', 'crowddispersal_predicted',
       'crowddispersal_probability', 'ignore_predicted', 'ignore_probability',
       'killings_predicted', 'killings_probability', 'shootings_predicted',
       'shootings_probability', 'violence_predicted', 'violence_probability'], inplace=True)

_Saving `mass_no_country` as CSV and pickle_

In [265]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [267]:
mass_no_country.to_csv('../data/03_mass_no_country.csv', index=False)

In [268]:
mass_no_country.to_pickle('../data/mass_no_country.pickle')